# Importing Libraries

In [11]:
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer 
import math
import nltk
import re
import requests
import bs4
!pip install contractions
import contractions
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

     |████████████████████████████████| 286kB 2.8MB/s 
     |████████████████████████████████| 327kB 4.7MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.2-cp37-cp37m-linux_x86_64.whl size=85406 sha256=b524fad4396c7705da0168f61092695a016c4c33dab19ff7095128e076fbb97d
  Stored in directory: /root/.cache/pip/wheels/3a/03/34/77e3ece0bba8b86bfac88a79f923b36d805cad63caeba38842
Successfully built pyahocorasick
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

#1. Text Summarization Using TF-IDF



We used Extracted Summarization Method. In this keyphrases are selected which occurs in the document.

### Converting The Paragraph into Sentences

In [ ]:
def paragraphToSentence(paragraph):

    sentencesDict = {}
    sentencesList = sent_tokenize(paragraph)
    
    for i in range(len(sentencesList)):
        sentencesDict[i] = sentencesList[i].strip()
    
    return sentencesDict

In [ ]:

def cleaning(file):
  documentsDict = {}
  for key in file:
    clean_S = contractions.fix(file[key])
    clean_S = re.sub(r'[\!@#$%^&\*()\_+={}\:\;<>\?/\|\-/"\']*',"", clean_S)  #junk symbols
    clean_S = re.sub('\s+', ' ', clean_S)    
    clean_S=re.sub("[^a-zA-Z\s\n\.]", " ",clean_S)  #used to eliminate non-ascii and numeric symbols 
    clean_S = re.sub(r"(.)\1{2}","",clean_S)  #for elongated words
    clean_S = clean_S.strip()
    documentsDict[key]=clean_S
  
  return documentsDict

In [ ]:
def clean(file):
  clean_S = contractions.fix(file)
  clean_S = re.sub("\[\d+\]","",clean_S).strip()
  clean_S = re.sub(r'[\!@#$%^&\*()\_+={}\,\:\;<>\?/\|\-/"\']*',"", clean_S)
  clean_S = re.sub('\s+', ' ', clean_S)    
  clean_S=re.sub("[^a-zA-Z0-9\s\n\.]", " ",clean_S)  #used to eliminate non-ascii and numeric symbols 
  clean_S = clean_S.strip()
  return clean_S

### Text Preprocessing
- Converting to Lowercase
- Tokenization & Punctuation Removal
- Removing Stopwords
- Lemmatization

In [ ]:
def preprocessing(sentenceDict):
    
    tokensDict = {}
    
    #Converting to Lowercase
    for sentenceNo in sentenceDict:
        sentence = sentenceDict[sentenceNo]
        tokensDict[sentenceNo] = sentence.lower()
    
    #Tokenization
    tokenizer = RegexpTokenizer(r'\w+')
    for sentenceNo in tokensDict:   
        sentence = tokensDict[sentenceNo]
        tokenSentenceList = tokenizer.tokenize(sentence)
        tokensDict[sentenceNo] = tokenSentenceList
            
    #Removing Stopwords
    stopwordsList = stopwords.words('english')
    
    for sentenceNo in tokensDict:
        sentenceTokenList = tokensDict[sentenceNo]

        for word in sentenceTokenList.copy():
            # print(word)
            if word in stopwordsList:
                sentenceTokenList.remove(word)
            
            if word.isdigit():
              sentenceTokenList.remove(word)
            tokensDict[sentenceNo] = sentenceTokenList
            
    #Lemmatization        
    lemmatizer = WordNetLemmatizer() 
    for sentenceNo in tokensDict:
        tokenOutputList = []
        sentenceTokenList = tokensDict[sentenceNo]

        for word in sentenceTokenList:
            tokenOutputList.append(lemmatizer.lemmatize(word))
        tokensDict[sentenceNo] = tokenOutputList

            
    return tokensDict

### Creating Word Frequency Table

In [ ]:
def frequenctTable(tokensDict):
    
    for sentenceNo in tokensDict:
        frequencyDict = {}
        sentenceTokenList = tokensDict[sentenceNo]
    
        for word in sentenceTokenList:
            if word not in frequencyDict:
                frequencyDict[word] = 1
            else:
                frequencyDict[word] += 1
        tokensDict[sentenceNo] = frequencyDict


    return tokensDict


### Calculating Term Frequency (TF)


In [ ]:
def termFrequency(frequencyTable):
    
    tfScoresDict = {}
    for sentenceNo in frequencyTable:
        frequencyDict = {}
        sentenceTokenDict = frequencyTable[sentenceNo]
    
        for term in sentenceTokenDict:
            frequencyDict[term] =  frequencyTable[sentenceNo][term] / len(sentenceTokenDict)

        tfScoresDict[sentenceNo] = frequencyDict

    return tfScoresDict

### Calculating Number of Sentences Containing a Word


In [ ]:
def sentenceContainingWord(frequencyTable):
    
    sentenceWordTable = {}
    
    for sentenceNo in frequencyTable:
        sentenceTokenDict = frequencyTable[sentenceNo]
        
        for word in sentenceTokenDict:
            if word not in sentenceWordTable:
                sentenceWordTable[word] = 1
            else:
                sentenceWordTable[word] += 1
    
    return sentenceWordTable

### Calculating Invese Document Frequency (IDF)

In [ ]:
def inverseDf(frequencyTable, sentencesWordDict):
    
    inverseDfScores = {}
    for sentenceNo in frequencyTable:
        idfDict = {}
        sentenceTokenDict = frequencyTable[sentenceNo]

        for term in sentenceTokenDict:
            idfDict[term] = math.log10(len(frequencyTable)/ sentencesWordDict[term])
            
        inverseDfScores[sentenceNo] = idfDict
        
    return inverseDfScores

### Calculating TF-IDF

In [ ]:
def calculateTfIdf(tfScores, inverseDfScores):
    
    tfidfScores = {}
        
    for sentenceNo in tfScores:
        tfidfDict = {}
        
        tfDict = tfScores[sentenceNo]
        idfDict = inverseDfScores[sentenceNo]
        
        for tf, idf in zip(tfDict, idfDict):
            tfidfDict[tf] = tfDict[tf] * idfDict[idf]
            
        tfidfScores[sentenceNo] = tfidfDict
        
    return tfidfScores

### Scoring Each Sentence

In [ ]:
def sentenceScoring(tfIdfScores):
    scoresList = []
    
    for sentenceNo in tfIdfScores:
        tfIdfScoresDict = tfIdfScores[sentenceNo]
        scoresList.append(sum(list(tfIdfScoresDict.values())) / len(tfIdfScoresDict))
    
    return scoresList


### Generating Summary

In [ ]:
def generatingSummary(sentenceDict, sentenceScores, threshold):
    
    summarySentences = []
    for scores in sentenceScores:
        if scores > threshold:
            summarySentences.append(sentenceScores.index(scores))
            
    summary = ''
    if summarySentences:
        for sentenceNo in summarySentences:
            summary += sentenceDict[sentenceNo] +" "
            
    return summary.strip()
  

In [ ]:
def getSummary(paragraph):
    sentenceDict = paragraphToSentence(paragraph)
    #print('Sentence Dictionary', sentenceDict)
    documentsDict = cleaning(sentenceDict)
    
    tokensDict = preprocessing(sentenceDict)
    #print('Tokens Dictionary', tokensDict)

    frequencyTable = frequenctTable(tokensDict)
    #print('Frequency table', frequencyTable)
    
    tfScores = termFrequency(frequencyTable)
    #print('TF Scores', tfScores)
    
    sentencesWordDict = sentenceContainingWord(frequencyTable)
    #print('Sentence Word Dictionary', sentencesWordDict)

    idfScores = inverseDf(frequencyTable, sentencesWordDict)
    #print('IDF Scores', idfScores)
    
    tfIdfScores = calculateTfIdf(tfScores, idfScores)
    #print('TF IDF Scores', tfIdfScores)
    
    sentenceScores = sentenceScoring(tfIdfScores)
    #print('Sentence Scores', sentenceScores)
    
    threshold = sum(sentenceScores) / len(sentenceDict)
    #print('Threshold', threshold)
    
    summary = generatingSummary(documentsDict, sentenceScores, threshold)
    return summary

In [ ]:
res = requests.get("https://en.wikipedia.org/wiki/Car")
wiki = bs4.BeautifulSoup(res.text, "lxml")
para = [''.join([i.getText() for i in wiki.select('p')])]
# para[0]
summary = getSummary(para[0])
print(summary)

A car or automobile is a wheeled motor vehicle used for transportation. Cars came into global use during theth century  and developed economies depend on them. Cars became widely available in the earlyth century. There are costs and benefits to car use. Traffic collisions are the largest because of injuryrelated deaths worldwide. The personal benefits include ondemand transportation  mobility  independence  and convenience. Peoples ability to move flexibly from place to place has farreaching implications for the nature of societies. There are aroundbillion cars in use worldwide. The numbers are increasing rapidly  especially in China  India and other newly industrialized countries. It originally referred to any wheeled horsedrawn vehicle  such as a cart  carriage  or wagon. Motor car  attested from is the usual formal term in British English. Autocar  a variant likewise attested fromand literally meaning selfpropelled car  is now considered archaic. Horseless carriage is attested from 

#2. Text Summarization Graph Based Method - **TextRank**

In [ ]:
from gensim.summarization.summarizer import summarize

In [ ]:
text = '''    
      Those Who Are Resilient Stay In The Game Longer
  “On the mountains of truth you can never climb in vain: either you will reach a point higher up today, or you will be training your powers so that you will be able to climb higher tomorrow.” — Friedrich Nietzsche
  Challenges and setbacks are not meant to defeat you, but promote you. However, I realise after many years of defeats, it can crush your spirit and it is easier to give up than risk further setbacks and disappointments. Have you experienced this before? To be honest, I don’t have the answers. I can’t tell you what the right course of action is; only you will know. However, it’s important not to be discouraged by failure when pursuing a goal or a dream, since failure itself means different things to different people. To a person with a Fixed Mindset failure is a blow to their self-esteem, yet to a person with a Growth Mindset, it’s an opportunity to improve and find new ways to overcome their obstacles. Same failure, yet different responses. Who is right and who is wrong? Neither. Each person has a different mindset that decides their outcome. Those who are resilient stay in the game longer and draw on their inner means to succeed.
  I’ve coached mummy and mom clients who gave up after many years toiling away at their respective goal or dream. It was at that point their biggest breakthrough came. Perhaps all those years of perseverance finally paid off. It was the 19th Century’s minister Henry Ward Beecher who once said: “One’s best success comes after their greatest disappointments.” No one knows what the future holds, so your only guide is whether you can endure repeated defeats and disappointments and still pursue your dream. Consider the advice from the American academic and psychologist Angela Duckworth who writes in Grit: The Power of Passion and Perseverance: “Many of us, it seems, quit what we start far too early and far too often. Even more than the effort a gritty person puts in on a single day, what matters is that they wake up the next day, and the next, ready to get on that treadmill and keep going.”
  I know one thing for certain: don’t settle for less than what you’re capable of, but strive for something bigger. Some of you reading this might identify with this message because it resonates with you on a deeper level. For others, at the end of their tether the message might be nothing more than a trivial pep talk. What I wish to convey irrespective of where you are in your journey is: NEVER settle for less. If you settle for less, you will receive less than you deserve and convince yourself you are justified to receive it.
  “Two people on a precipice over Yosemite Valley” by Nathan Shipps on Unsplash
  Develop A Powerful Vision Of What You Want
  “Your problem is to bridge the gap which exists between where you are now and the goal you intend to reach.” — Earl Nightingale
  I recall a passage my father often used growing up in 1990s: “Don’t tell me your problems unless you’ve spent weeks trying to solve them yourself.” That advice has echoed in my mind for decades and became my motivator. Don’t leave it to other people or outside circumstances to motivate you because you will be let down every time. It must come from within you. Gnaw away at your problems until you solve them or find a solution. Problems are not stop signs, they are advising you that more work is required to overcome them. Most times, problems help you gain a skill or develop the resources to succeed later. So embrace your challenges and develop the grit to push past them instead of retreat in resignation. Where are you settling in your life right now? Could you be you playing for bigger stakes than you are? Are you willing to play bigger even if it means repeated failures and setbacks? You should ask yourself these questions to decide whether you’re willing to put yourself on the line or settle for less. And that’s fine if you’re content to receive less, as long as you’re not regretful later.
  If you have not achieved the success you deserve and are considering giving up, will you regret it in a few years or decades from now? Only you can answer that, but you should carve out time to discover your motivation for pursuing your goals. It’s a fact, if you don’t know what you want you’ll get what life hands you and it may not be in your best interest, affirms author Larry Weidel: “Winners know that if you don’t figure out what you want, you’ll get whatever life hands you.” The key is to develop a powerful vision of what you want and hold that image in your mind. Nurture it daily and give it life by taking purposeful action towards it.
  Vision + desire + dedication + patience + daily action leads to astonishing success. Are you willing to commit to this way of life or jump ship at the first sign of failure? I’m amused when I read questions written by millennials on Quora who ask how they can become rich and famous or the next Elon Musk. Success is a fickle and long game with highs and lows. Similarly, there are no assurances even if you’re an overnight sensation, to sustain it for long, particularly if you don’t have the mental and emotional means to endure it. This means you must rely on the one true constant in your favour: your personal development. The more you grow, the more you gain in terms of financial resources, status, success — simple. If you leave it to outside conditions to dictate your circumstances, you are rolling the dice on your future.
  So become intentional on what you want out of life. Commit to it. Nurture your dreams. Focus on your development and if you want to give up, know what’s involved before you take the plunge. Because I assure you, someone out there right now is working harder than you, reading more books, sleeping less and sacrificing all they have to realise their dreams and it may contest with yours. Don’t leave your dreams to chance.

    '''

summaryList = summarize(text, ratio = 0.05, split = '\n')

print('\n'.join(summaryList))

It was the 19th Century’s minister Henry Ward Beecher who once said: “One’s best success comes after their greatest disappointments.” No one knows what the future holds, so your only guide is whether you can endure repeated defeats and disappointments and still pursue your dream.
I know one thing for certain: don’t settle for less than what you’re capable of, but strive for something bigger.


In [ ]:
res = requests.get("https://en.wikipedia.org/wiki/Car")
wiki = bs4.BeautifulSoup(res.text, "lxml")
para = [''.join([i.getText() for i in wiki.select('p')])]
# para[0]

summaryList = summarize(para[0], ratio = 0.05, split = '\n')

print('\n'.join(summaryList))


### BERT 

In [ ]:
!pip install bert-extractive-summarizer
!pip install transformers==2.2.0
!pip install spacy==2.0.12

!pip uninstall thinc
!pip uninstall cymem
!pip install spacy

     |████████████████████████████████| 133kB 8.4MB/s 
  Found existing installation: cymem 1.31.2
    Uninstalling cymem-1.31.2:
      Successfully uninstalled cymem-1.31.2
  Found existing installation: preshed 1.0.1
    Uninstalling preshed-1.0.1:
      Successfully uninstalled preshed-1.0.1


Processing /root/.cache/pip/wheels/60/0b/bb/7c2e28db574dbb2358176934eddd32a1c5f838ba0bc23eaaab/spacy-2.0.12-cp37-cp37m-linux_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/17/4e/50e8e4cf5f00b537095711c2c86ac4d7191aed2b4fffd5a19f06898f6929/ujson-4.0.2-cp37-cp37m-manylinux1_x86_64.whl
Processing /root/.cache/pip/wheels/75/07/38/3c16b529d50cb4e0cd3dbc7b75cece8a09c132692c74450b01/regex-2017.4.5-cp37-cp37m-linux_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/94/b1/47a88072d0a38b3594c0a638a62f9ef7c742b8b8a87f7b105f7ed720b14b/thinc-6.10.3.tar.gz
  Using cached https://files.pythonhosted.org/packages/f4/0c/64933c18f02fbf46acdd0c7705ec1c1194f58b564bb5a2d140fabcb37bad/preshed-1.0.1-cp37-cp37m-manylinux1_x86_64.whl
Processing /root/.cache/pip/wheels/55/8d/4a/f6328252aa2aaec0b1cb906fd96a1566d77f0f67701071ad13/cymem-1.31.2-cp37-cp37m-linux_x86_64.whl
  ERROR: Failed building wheel for thinc
  Running setup.py clean for thinc
Failed to build thinc
ERROR: fastai 

In [ ]:
from summarizer import Summarizer,TransformerSummarizer
import transformers
from transformers import XLMWithLMHeadModel, XLMTokenizer

In [ ]:
text = '''    
      Those Who Are Resilient Stay In The Game Longer
  “On the mountains of truth you can never climb in vain: either you will reach a point higher up today, or you will be training your powers so that you will be able to climb higher tomorrow.” — Friedrich Nietzsche
  Challenges and setbacks are not meant to defeat you, but promote you. However, I realise after many years of defeats, it can crush your spirit and it is easier to give up than risk further setbacks and disappointments. Have you experienced this before? To be honest, I don’t have the answers. I can’t tell you what the right course of action is; only you will know. However, it’s important not to be discouraged by failure when pursuing a goal or a dream, since failure itself means different things to different people. To a person with a Fixed Mindset failure is a blow to their self-esteem, yet to a person with a Growth Mindset, it’s an opportunity to improve and find new ways to overcome their obstacles. Same failure, yet different responses. Who is right and who is wrong? Neither. Each person has a different mindset that decides their outcome. Those who are resilient stay in the game longer and draw on their inner means to succeed.
  I’ve coached mummy and mom clients who gave up after many years toiling away at their respective goal or dream. It was at that point their biggest breakthrough came. Perhaps all those years of perseverance finally paid off. It was the 19th Century’s minister Henry Ward Beecher who once said: “One’s best success comes after their greatest disappointments.” No one knows what the future holds, so your only guide is whether you can endure repeated defeats and disappointments and still pursue your dream. Consider the advice from the American academic and psychologist Angela Duckworth who writes in Grit: The Power of Passion and Perseverance: “Many of us, it seems, quit what we start far too early and far too often. Even more than the effort a gritty person puts in on a single day, what matters is that they wake up the next day, and the next, ready to get on that treadmill and keep going.”
  I know one thing for certain: don’t settle for less than what you’re capable of, but strive for something bigger. Some of you reading this might identify with this message because it resonates with you on a deeper level. For others, at the end of their tether the message might be nothing more than a trivial pep talk. What I wish to convey irrespective of where you are in your journey is: NEVER settle for less. If you settle for less, you will receive less than you deserve and convince yourself you are justified to receive it.
  “Two people on a precipice over Yosemite Valley” by Nathan Shipps on Unsplash
  Develop A Powerful Vision Of What You Want
  “Your problem is to bridge the gap which exists between where you are now and the goal you intend to reach.” — Earl Nightingale
  I recall a passage my father often used growing up in 1990s: “Don’t tell me your problems unless you’ve spent weeks trying to solve them yourself.” That advice has echoed in my mind for decades and became my motivator. Don’t leave it to other people or outside circumstances to motivate you because you will be let down every time. It must come from within you. Gnaw away at your problems until you solve them or find a solution. Problems are not stop signs, they are advising you that more work is required to overcome them. Most times, problems help you gain a skill or develop the resources to succeed later. So embrace your challenges and develop the grit to push past them instead of retreat in resignation. Where are you settling in your life right now? Could you be you playing for bigger stakes than you are? Are you willing to play bigger even if it means repeated failures and setbacks? You should ask yourself these questions to decide whether you’re willing to put yourself on the line or settle for less. And that’s fine if you’re content to receive less, as long as you’re not regretful later.
  If you have not achieved the success you deserve and are considering giving up, will you regret it in a few years or decades from now? Only you can answer that, but you should carve out time to discover your motivation for pursuing your goals. It’s a fact, if you don’t know what you want you’ll get what life hands you and it may not be in your best interest, affirms author Larry Weidel: “Winners know that if you don’t figure out what you want, you’ll get whatever life hands you.” The key is to develop a powerful vision of what you want and hold that image in your mind. Nurture it daily and give it life by taking purposeful action towards it.
  Vision + desire + dedication + patience + daily action leads to astonishing success. Are you willing to commit to this way of life or jump ship at the first sign of failure? I’m amused when I read questions written by millennials on Quora who ask how they can become rich and famous or the next Elon Musk. Success is a fickle and long game with highs and lows. Similarly, there are no assurances even if you’re an overnight sensation, to sustain it for long, particularly if you don’t have the mental and emotional means to endure it. This means you must rely on the one true constant in your favour: your personal development. The more you grow, the more you gain in terms of financial resources, status, success — simple. If you leave it to outside conditions to dictate your circumstances, you are rolling the dice on your future.
  So become intentional on what you want out of life. Commit to it. Nurture your dreams. Focus on your development and if you want to give up, know what’s involved before you take the plunge. Because I assure you, someone out there right now is working harder than you, reading more books, sleeping less and sacrificing all they have to realise their dreams and it may contest with yours. Don’t leave your dreams to chance.

    '''

In [ ]:
Bert_model = Summarizer()
def bert_fun(Bert_model,text):
  bert_summary = ''.join(Bert_model(text, min_length=60))
  return bert_summary
sum_bert = bert_fun(Bert_model,text)
sum_bert

'Those Who Are Resilient Stay In The Game Longer\n  “On the mountains of truth you can never climb in vain: either you will reach a point higher up today, or you will be training your powers so that you will be able to climb higher tomorrow.” To a person with a Fixed Mindset failure is a blow to their self-esteem, yet to a person with a Growth Mindset, it’s an opportunity to improve and find new ways to overcome their obstacles. Those who are resilient stay in the game longer and draw on their inner means to succeed. “Two people on a precipice over Yosemite Valley” by Nathan Shipps on Unsplash\n  Develop A Powerful Vision Of What You Want\n  “Your problem is to bridge the gap which exists between where you are now and the goal you intend to reach.” Gnaw away at your problems until you solve them or find a solution. You should ask yourself these questions to decide whether you’re willing to put yourself on the line or settle for less. And that’s fine if you’re content to receive less, a

In [ ]:
def bert_fun(Bert_model,text):
  bert_summary = ''.join(Bert_model(text, min_length=60,max_length=300))
  return bert_summary
sum_bert = bert_fun(Bert_model,text)
sum_bert

#3. Abstractive Text Summarization

### XLNet

In [ ]:
text = clean(para[0])

In [ ]:
XLNet_model = TransformerSummarizer(transformer_type="XLNet",transformer_model_key="xlnet-base-cased") #XLNet Model
def XLNet_fun(XLNet_model,text):
  full_text = ''.join(XLNet_model(text, min_length=60,max_length=400))
  return full_text
sum_XLNet = XLNet_fun(XLNet_model,text)
sum_XLNet

100%|██████████| 798011/798011 [00:00<00:00, 1835530.42B/s]


'A car (or automobile) is a wheeled motor vehicle used for transportation. It was unable to maintain sufficient steam pressure for long periods and was of little practical use. A variety of steam-powered road vehicles were used during the first part of the 19th century, including steam cars, steam buses, phaetons, and steam rollers. Many of his other inventions made the use of the internal combustion engine feasible for powering a vehicle. During the last years of the nineteenth century, Benz was the largest car company in the world with 572 units produced in 1899 and, because of its size, Benz & Cie., Benz, Maybach and the Daimler team seem to have been unaware of each other\'s early work. This was a production of a small number of vehicles for Jellinek to race and market in his country. Rights to the Daimler brand name were sold to other manufacturers. In 1891, Auguste Doriot and his Peugeot colleague Louis Rigoulot completed the longest trip by a gasoline-powered vehicle when their 

In [ ]:
def XLNet_fun(XLNet_model,text):
  full_text = ''.join(XLNet_model(text, min_length=60,max_length=150))
  return full_text
sum_XLNet = XLNet_fun(XLNet_model,text)
sum_XLNet

'A car or automobile is a wheeled motor vehicle used for transportation. Traffic collisions are the largest because of injuryrelated deaths worldwide. Autocar a variant likewise attested from 1895 and literally meaning selfpropelled car is now considered archaic. He was awarded the patent for its invention as of his application on 29 January 1886 under the auspices of his major company Benz Cie. Rights to the Daimler brand name were sold to other manufacturers. The first design for an American car with a gasoline internal combustion engine was made in 1877 by George Selden of Rochester New York. In 1892 German engineer Rudolf Diesel was granted a patent for a New Rational Combustion Engine. Steam electric and gasolinepowered vehicles competed for decades with gasoline internal combustion engines achieving dominance in the 1910s. This is the source of Fords apocryphal remark any color as long as it is black. The efficiency gains from the assembly line also coincided with the economic ri

### GPT - 2

In [ ]:
def gpt2(gpt2Model, textAfterPreprocessing):
  summary = (gpt2Model(textAfterPreprocessing, min_length=60))
  print(summary)

#Keep me global 
gpt2Model = TransformerSummarizer(transformer_type = 'GPT2', transformer_model_key = 'gpt2-medium')

sum_gpt2 = gpt2(gpt2Model, text)
sum_gpt2

Those Who Are Resilient Stay In The Game Longer
  “On the mountains of truth you can never climb in vain: either you will reach a point higher up today, or you will be training your powers so that you will be able to climb higher tomorrow.” To a person with a Fixed Mindset failure is a blow to their self-esteem, yet to a person with a Growth Mindset, it’s an opportunity to improve and find new ways to overcome their obstacles. Each person has a different mindset that decides their outcome. No one knows what the future holds, so your only guide is whether you can endure repeated defeats and disappointments and still pursue your dream. What I wish to convey irrespective of where you are in your journey is: NEVER settle for less. — Earl Nightingale
  I recall a passage my father often used growing up in 1990s: “Don’t tell me your problems unless you’ve spent weeks trying to solve them yourself.” Gnaw away at your problems until you solve them or find a solution. And that’s fine if you’re 

### XLM Transformer

In [ ]:
res = requests.get("https://en.wikipedia.org/wiki/Car")
wiki = bs4.BeautifulSoup(res.text, "lxml")
para = [''.join([i.getText() for i in wiki.select('p')])]


In [ ]:
text = clean(para[0])

In [ ]:
XLM_model = XLMWithLMHeadModel.from_pretrained('xlm-mlm-en-2048') #XLM Model
XLM_model.config.max_length = 512
tokenizer = XLMTokenizer.from_pretrained('xlm-mlm-en-2048')

100%|██████████| 486639/486639 [00:00<00:00, 20208028.36B/s]


In [ ]:
def XLM_Transformer_fun(text,model):
  inputs = tokenizer.batch_encode_plus([text], return_tensors='pt',max_length=511)
  sum_xlm_ids = model.generate(inputs['input_ids'])
  sum_xlm = tokenizer.decode(sum_xlm_ids[0], skip_special_tokens=True)
  return sum_xlm
sum_xlm = XLM_Transformer_fun(text,XLM_model)
sum_xlm

# Question Answering

In [ ]:
from transformers import pipeline

In [ ]:
!pip install git+https://github.com/huggingface/transformers

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-x7n6yeht
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-x7n6yeht
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for transformers: filename=transformers-4.6.0.dev0-cp37-none-any.whl size=2213074 sha256=cc9b87bad4e4ab6d211ae2e41c8beb3ca16e89939921778c4c72f878ada84999
  Stored in directory: /tmp/pip-ephem-wheel-cache-zcls1gdq/wheels/70/d3/52/b3fa4f8b8ef04167ac62e5bb2accb62ae764db2a378247490e
Successfully built transformers
  Found existing installation: transformers 2.2.0
    Uninstalling transformers-2.2.0:
      Successfully uninstalled transformers-2.2.0


In [ ]:
question = "What is the capital of the Netherlands?"
context = r"The four largest cities in the Netherlands are Amsterdam, Rotterdam, The Hague and Utrecht.[17] Amsterdam is the country's most populous city and nominal capital,[18] while The Hague holds the seat of the States General, Cabinet and Supreme Court.[19] The Port of Rotterdam is the busiest seaport in Europe, and the busiest in any country outside East Asia and Southeast Asia, behind only China and Singapore."

# Generating an answer to the question in context
qa = pipeline("question-answering")
answer = qa(question=question, context=context)

# Print the answer
print(f"Question: {question}")
print(f"Answer: '{answer['answer']}' with score {answer['score']}")


Question: What is the capital of the Netherlands?
Answer: 'Amsterdam' with score 0.3774995803833008


In [ ]:
question_answerer = pipeline('question-answering')
question_answerer({
    'question': 'What is the Newtons third law of motion?',
    'context': 'Newton’s third law of motion states that, "For every action there is equal and opposite reaction"'})

{'answer': '"For every action there is equal and opposite reaction"',
 'end': 97,
 'score': 0.6062516570091248,
 'start': 42}

In [ ]:
question_answerer({
    'question': 'What is blackhole?',
    'context': 'Newton’s third law of motion states that, "For every action there is equal and opposite reaction"'})

{'answer': 'For every action there is equal and opposite reaction',
 'end': 96,
 'score': 0.16732703149318695,
 'start': 43}

In [ ]:
question = "What is the capital of the India?"
context = r"The four largest cities in the Netherlands are Amsterdam, Rotterdam, The Hague and Utrecht.[17] Amsterdam is the country's most populous city and nominal capital,[18] while The Hague holds the seat of the States General, Cabinet and Supreme Court.[19] The Port of Rotterdam is the busiest seaport in Europe, and the busiest in any country outside East Asia and Southeast Asia, behind only China and Singapore."

In [ ]:
qa = pipeline("question-answering")
answer = qa(question=question, context=context)
print(f"Question: {question}")
print(f"Answer: '{answer['answer']}' with score {answer['score']}")

Question: What is the capital of the India?
Answer: 'China and Singapore' with score 0.324933260679245


In [ ]:
answer['answer']

'China and Singapore'

In [ ]:
# question = request.json['query']
# context = request.json['paragraph']
qa = pipeline("question-answering")
answer = qa(question=question, context=context)

output = {
          'result': answer['answer']
          }

In [ ]:
from flask import request, jsonify
import requests
import bs4

In [ ]:
output = {'acd' : 'xyz' }

In [ ]:
res = requests.get("https://en.wikipedia.org/wiki/"+'Car')
wiki = bs4.BeautifulSoup(res.text, "lxml")
para = [''.join([i.getText() for i in wiki.select('p')])]
summary = getSummary(para[0])